In [ ]:
language = 'pt-br'

In [ ]:
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async () =>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  js_result = output.eval_js('record(%s)' % (sec *1000))
  audio = b64decode(js_result.split(',')[1])
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)
  return f'/content/{file_name}'

print("Ouvindo...\n")
record_file = record()
display(Audio(record_file, autoplay=True))

Ouvindo...



<IPython.core.display.Javascript object>

In [ ]:
!pip -q install git+https://github.com/openai/whisper.git
!apt-get -qq update && apt-get -qq install -y ffmpeg

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 5.8 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
import whisper

language = "pt"
model = whisper.load_model("small")  # CPU-friendly
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(transcription)

 O que é o Youtube?


In [ ]:
!pip install openai

In [ ]:
!pip -q install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.2 MB/s eta 0:00:00


In [ ]:
# --- LLM open-source (100% grátis) ---

!pip -q install transformers accelerate sentencepiece

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Modelo leve e gratuito (bom para estudo)
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"



tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map={"": "cpu"}
)

def local_assistant_reply(user_text: str) -> str:
    system_prompt = (
        "Você é um assistente de voz em português. "
        "Responda de forma curta, clara e útil."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_text}
    ]

    # Usa o template de chat do modelo (quando disponível)
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=180,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Pega só a parte final (resposta do assistente)
    # Como cada modelo “formata” diferente, esse corte é simples e robusto:
    return decoded.split(messages[-1]["content"])[-1].strip()

reply = local_assistant_reply(transcription)
print(reply)

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

<|assistant|>
O Youtube é um serviço online de vídeos, onde você pode ler, ver e reproduzir vídeos em qualquer lugar e tempo, sem precisar ter uma rede de internet. O Youtube foi criado em 2005 e atualmente possui mais de 1 bilhão de afiliados. 

Se você estiver interessado em encontrar vídeos de conteúdo de todo tipo, como entrevistas, comédias, filmes, documentários, música, entre outros, o Youtube é um lugar para você encontrar e assistir a esses vídeos.


In [ ]:
!pip install gTTS

In [ ]:
from gtts import gTTS
from IPython.display import Audio, display

gtts_object = gTTS(text=reply, lang=language, slow=False)
response_audio = "/content/response_audio.mp3"  # gTTS salva em MP3
gtts_object.save(response_audio)

print("Falando...\n")
display(Audio(response_audio, autoplay=True))


Falando...



In [ ]:
#Localizar o arquivo
import os, glob

print("Arquivos .ipynb em /content:")
for p in glob.glob("/content/*.ipynb"):
    print(" -", p)

print("\nArquivos .ipynb no Drive (se montado):")
for p in glob.glob("/content/drive/MyDrive/**/*.ipynb", recursive=True)[:50]:
    print(" -", p)

Arquivos .ipynb em /content:

Arquivos .ipynb no Drive (se montado):
 - /content/drive/MyDrive/Colab Notebooks/desafio_assistente_whisper.ipynb


In [ ]:
import json, shutil, time

path = "/content/drive/MyDrive/Colab Notebooks/desafio_assistente_whisper.ipynb"

# backup
backup = path.replace(".ipynb", f".bak-{int(time.time())}.ipynb")
shutil.copy2(path, backup)

with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)

# REMOVE widgets do metadata do notebook (isso resolve o GitHub)
data.get("metadata", {}).pop("widgets", None)

# (opcional) remove widgets de metadados das células também
for cell in data.get("cells", []):
    if isinstance(cell, dict):
        cell.get("metadata", {}).pop("widgets", None)

with open(path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=1)

# valida
with open(path, "r", encoding="utf-8") as f:
    data2 = json.load(f)

print("Backup:", backup)
print("Tem metadata.widgets agora?", "widgets" in data2.get("metadata", {}))

Backup: /content/drive/MyDrive/Colab Notebooks/desafio_assistente_whisper.bak-1772106934.ipynb
Tem metadata.widgets agora? False
